### Installazione e Import Librerie

In [ ]:
!pip install spacy-dbpedia-spotlight
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf
!pip install spacy[transformers]
!pip install nervaluate
!pip install SPARQLWrapper

In [2]:
import pandas as pd
import re
import itertools
import spacy
import spacy_dbpedia_spotlight
import spacy
import pandas as pd
from spacy import displacy
import ast
import numpy as np
import urllib
import traceback
import sys

### Preprocessing

In [3]:
entity_list = list(pd.read_csv('characters_list.csv', sep=";")['Entity'])
# entity_list = ['Dumbledore', 'Hermione', 'Harry', 'McGonagall', 'Ron', 'Voldemort']

In [4]:
books = ["Book1", "Book2", "Book3", "Book4", "Book5", "Book6", "Book7"]
titles = ["Philosophers Stone", "Chamber of Secrets", "Prisoner of Azkaban", "Goblet of Fire",
          "Order of the Phoenix", "Half Blood Prince", "Deathly Hallows"]

In [5]:
all_lines = ""
for book, title in zip(books, titles):
  print(f"processing {book}")
  with open(f'{book}.txt') as f:
      lines = f.read().strip('\n').replace('\n', ' ')

  # miglioramenti
  lines = lines.replace("Mr.", "Mr")
  lines = lines.replace("Mrs.", "Mrs")
  lines = lines.replace("...", "")

  # rimuovere l'indice del capitolo 
  lines = re.sub(f'Page | \d+ Harry Potter and the {title} - J.K. Rowling', '', lines)
  lines = lines.replace("|", "")
  all_lines = all_lines + lines

processing Book1
processing Book2
processing Book3
processing Book4
processing Book5
processing Book6
processing Book7


In [6]:
# creazione della lista delle frasi
phrases_list = all_lines.split(".")

In [7]:
results = []

for entity in entity_list:
  cont = 2
  entity_phrases = []
  for i in phrases_list:
    if cont == 0:
      break
    if entity in i and 200 < len(i) < 1000:
      entity_phrases.append([entity, i])
      cont -= 1

  results.append(entity_phrases)

In [8]:
data = list(itertools.chain(*results))

In [9]:
len(data)

111

In [10]:
df = pd.DataFrame(data, columns=['entity', 'phrase'])

In [11]:
df.head(10)

,entity,phrase
0,Regulus,"The Dark Mark had been set over it — well, f..."
1,Regulus,"It was a pompous little sign, neatly lettere..."
2,Sirius,” Harry held the paper up to the candlelight...
3,Sirius,Harry spent the long sunny days explor...
4,Cho,What she did have were Bertie Bott’s Every F...
5,Cho,"Choose, unless you wish to stay here foreve..."
6,Crabbe,Scabbers the rat was hanging off his finger...
7,Crabbe,” Malfoy swaggered forward when his name was...
8,Mr Crouch,"” “Oh Bagman’s likable enough, of course,” s..."
9,Mr Crouch,Mr Crouch has been taking a personal interes...


In [12]:
df.to_csv('5_phrases_for_entity_hp.csv')

In [13]:
df = pd.read_csv("5_phrases_for_entity_hp.csv", sep=",", header=None)
df.columns = ["index", "entity", "phrase"]
df = df.iloc[1:]
df = df.drop(columns=["index"])

In [14]:
df

,entity,phrase
1,Regulus,"The Dark Mark had been set over it — well, f..."
2,Regulus,"It was a pompous little sign, neatly lettere..."
3,Sirius,” Harry held the paper up to the candlelight...
4,Sirius,Harry spent the long sunny days explor...
5,Cho,What she did have were Bertie Bott’s Every F...
...,...,...
107,Myrtle,“Come to throw something else at me?” Harr...
108,Aragog,"” Aragog clicked his pincers furiously, and ..."
109,Aragog,For nearly a quarter of an hour he spoke int...
110,Grawp,"The back, under what looked like a dirty bro..."


## Named Entity Recognition per le frasi selezionate

In [15]:
nlp = spacy.load('en_core_web_lg')
# stampà le entità
for x in range(df.shape[0]):
    doc = nlp(df["phrase"].values[x])
    print(doc)
    displacy.render(doc, style="ent", jupyter=True)

 The Dark Mark  had been set over it — well, frankly, I’m surprised he  stayed alive for even a year after deserting the Death  Eaters; Sirius’s brother, Regulus, only managed a few  days as far as I can remember


 It  was a pompous little sign, neatly lettered by hand,  the sort of thing that Percy Weasley might have stuck  on his bedroom door:   Do Not Enter   Without the Express Permission of  Regulus Arcturus Black   Excitement trickled through Harry, but he was not  immediately sure why


”   Harry held the paper up to the candlelight and read:  BLACK STILL AT LARGE   Sirius Black, possibly the most infamous prisoner  ever to be held in Azkaban fortress, is still eluding  capture, the Ministry of Magic confirmed today


        Harry spent the long sunny days exploring the shops  and eating under the brightly colored umbrellas  outside cafes, where his fellow diners were showing  one another their purchases (“it’s a lunascope, old  boy — no more messing around with moon charts,  see?”) or else discussing the case of Sirius Black  (“personally, I won’t let any of the children out alone  until he’s back in Azkaban”)


 What she did have were Bertie Bott’s  Every Flavor Beans, Drooble’s Best Blowing Gum,  Chocolate Frogs, Pumpkin Pasties, Cauldron Cakes,  Licorice Wands, and a number of other strange things  Harry had never seen in his life


...

 “Come to  throw something else at me?”   Harry waded across to her stall and said, “Why would  I throw something at you?”   “Don’t ask me,” Myrtle shouted, emerging with a wave  of yet more water, which splashed onto the already  sopping floor


”   Aragog clicked his pincers furiously, and all around  the hollow the sound was echoed by the crowd of         spiders; it was like applause, except applause didn’t  usually make Harry feel sick with fear


 For nearly a  quarter of an hour he spoke into the rapt silence: He  told them about hearing the disembodied voice, how  Hermione had finally realized that he was hearing a  basilisk in the pipes; how he and Ron had followed  the spiders into the forest, that Aragog had told them  where the last victim of the basilisk had died; how he  had guessed that Moaning Myrtle had been the  victim, and that the entrance to the Chamber of  Secrets might be in her bathroom


 The back, under what  looked like a dirty brownish smock comprised of  animal skins sewn roughly together, was very broad,  and as Grawp slept, it seemed to strain a little at the  rough seams of the skins


 “An’ I don’ wan’ yeh ter  put yerself out too much, like I know yeh’ve got  exams If yeh could jus’ nip down here in yer  Invisibility Cloak maybe once a week an’ have a little  chat with him  I’ll wake him up, then — introduce  you — ”   “Wha — no!” said Hermione, jumping up, “Hagrid, no,  don’t wake him, really, we don’t need — ”   But Hagrid had already stepped over the great trunk  in front of them and was proceeding toward Grawp


### Selezione delle Label e creazione del dataframe per l'evaluation di NER

In [16]:
# caricamento dati
name_entity_label_risultati=[]
df_ner = pd.DataFrame()
df_ner["frase"] = df["phrase"]
df_ner["ner"] = 0
sentence = list(df["phrase"])
print(sentence)
contatore=1
lista_entità_person=[]
lista_entità_product=[]
lista_entità_org=[]
for i in sentence:
  x = []
  doc = nlp(i)
  for ent in doc.ents:
    x.append({"label" : ent.label_ ,  "start" : ent.start_char , "end" : ent.end_char})
    if(ent.label_ == "PERSON"):
      lista_entità_person.append(ent.text)
    elif(ent.label_ == "PRODUCT"):
      lista_entità_product.append(ent.text)
    elif(ent.label_ == "ORG"):
      lista_entità_org.append(ent.text)
  df_ner["ner"][contatore] = x
  name_entity_label_risultati.append(x)
  contatore = contatore + 1


print(df_ner["ner"])
print(name_entity_label_risultati)


[' The Dark Mark  had been set over it — well, frankly, I’m surprised he  stayed alive for even a year after deserting the Death  Eaters; Sirius’s brother, Regulus, only managed a few  days as far as I can remember', ' It  was a pompous little sign, neatly lettered by hand,  the sort of thing that Percy Weasley might have stuck  on his bedroom door:   Do Not Enter   Without the Express Permission of  Regulus Arcturus Black   Excitement trickled through Harry, but he was not  immediately sure why', '”   Harry held the paper up to the candlelight and read:  BLACK STILL AT LARGE   Sirius Black, possibly the most infamous prisoner  ever to be held in Azkaban fortress, is still eluding  capture, the Ministry of Magic confirmed today', '        Harry spent the long sunny days exploring the shops  and eating under the brightly colored umbrellas  outside cafes, where his fellow diners were showing  one another their purchases (“it’s a lunascope, old  boy — no more messing around with moon char

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


1      [{'label': 'PRODUCT', 'start': 1, 'end': 14}, ...
2      [{'label': 'PERSON', 'start': 81, 'end': 94}, ...
3      [{'label': 'ORG', 'start': 81, 'end': 93}, {'l...
4      [{'label': 'DATE', 'start': 20, 'end': 39}, {'...
5      [{'label': 'PERSON', 'start': 24, 'end': 35}, ...
                             ...                        
107      [{'label': 'PERSON', 'start': 141, 'end': 147}]
108        [{'label': 'PRODUCT', 'start': 4, 'end': 10}]
109    [{'label': 'DATE', 'start': 5, 'end': 22}, {'l...
110         [{'label': 'ORG', 'start': 132, 'end': 137}]
111    [{'label': 'PERSON', 'start': 18, 'end': 25}, ...
Name: ner, Length: 111, dtype: object
[[{'label': 'PRODUCT', 'start': 1, 'end': 14}, {'label': 'ORG', 'start': 136, 'end': 142}, {'label': 'DATE', 'start': 176, 'end': 187}], [{'label': 'PERSON', 'start': 81, 'end': 94}, {'label': 'ORG', 'start': 159, 'end': 181}], [{'label': 'ORG', 'start': 81, 'end': 93}, {'label': 'PRODUCT', 'start': 151, 'end': 158}, {'label': 'ORG',

In [17]:
myset = set(lista_entità_person)
print(myset)
lista_entità_person = list(myset)


{'Harry', 'Mr Crouch', 'Lucius Malfoy', 'Charlie', 'Bellatrix', 'Remembrall', 'Cedric Diggory', 'Percy Weasley', 'Weasley   ', 'Bagman', 'Aberforth', 'Dean', 'Mrs Weasley', 'Hermione', 'George Weasley', 'Umbridge', 'Nicolas  Flamel', 'Frank', 'Minerva', 'Harry Potter Day', 'Mr Malfoy', 'Mr Weasley', 'Quirrell', 'Fred', 'Voldemort', 'Arthur', 'Lily', 'Crabbe', 'Ginny', 'Uncle Vernon', 'Floo', 'Remus John Lupin', 'Narcissa', 'Draco Malfoy', 'Arthur Weasley', 'Harry Potter', 'Bill', 'Alice Longbottom', 'Hem', 'Wormtail', 'Seamus', 'chase Fred', 'Frank Longbottom', 'Moony', 'Delacour', 'Gryffindor', 'Stan', 'Neville Longbottom', 'Barty Crouch', 'Karkaroff', 'Luna Lovegood', 'Harry James Potter', 'Ron', 'Percy  ', 'Hermione Granger', 'Nicolas Flamel', 'Hedwig', 'Neville', 'Myrtle', 'Malfoy', 'Rowling      ', 'Bertie Bott', 'Percy', 'McGonagall', 'Aunt Petunia', 'Penelope', 'Maxime', 'Grindelwald', 'Dobby', 'Flitwick', 'Xenophilius Lovegood', 'George', 'yeh ter', 'Goyle', 'Cornelius Fudge', 

In [18]:
for i in lista_entità_person:
  i.replace(' +','')

In [19]:
df_ner.to_csv("ner.csv", sep=";")

#csv che verranno passati a Word2Vec e Cade
data = pd.DataFrame(data=lista_entità_person)
data= data.rename(columns={0: "person"})
data.to_csv("person_label.csv")

data2 = pd.DataFrame(data=lista_entità_product)
data2= data2.rename(columns={0: "product"})
data2.to_csv("product_label.csv")


data3 = pd.DataFrame(data=lista_entità_org)
data3= data3.rename(columns={0: "org"})
data3.to_csv("org_label.csv")


In [20]:
df_ner_corretto = pd.read_csv("ner_corretto_csv.csv",sep=";", header=None)
df_ner_corretto.columns = ["index", "phrase", "label"]
df_ner_corretto = df_ner_corretto.iloc[1: , 1:]
df_ner_corretto
df_ner_corretto_label = df_ner_corretto.label

In [21]:
print(df_ner_corretto_label)

1      [{'label': 'PRODUCT', 'start': 1, 'end': 14}, ...
2      [{'label': 'PERSON', 'start': 81, 'end': 94}, ...
3      [{'label': 'PERSON', 'start': 81, 'end': 93}, ...
4      [{'label': 'DATE', 'start': 20, 'end': 39}, {'...
5      [{'label': 'PERSON', 'start': 24, 'end': 35}, ...
                             ...                        
107      [{'label': 'PERSON', 'start': 141, 'end': 147}]
108         [{'label': ‘PERSON', 'start': 4, 'end': 10}]
109    [{'label': 'DATE', 'start': 5, 'end': 22}, {'l...
110      [{'label': ‘PERSON', 'start': 132, 'end': 137}]
111    [{'label': 'PRODUCT', 'start': 18, 'end': 25},...
Name: label, Length: 111, dtype: object


In [22]:
ner_corretto = []
for i in df_ner_corretto_label:
  if (len(i) > 2):
    i=i.replace("‘", "'")
    d = ast.literal_eval(i)
    ner_corretto.append(d)
ner_corretto


[[{'end': 14, 'label': 'PRODUCT', 'start': 1},
  {'end': 142, 'label': 'PERSON', 'start': 136},
  {'end': 187, 'label': 'DATE', 'start': 176}],
 [{'end': 94, 'label': 'PERSON', 'start': 81},
  {'end': 181, 'label': 'ORG', 'start': 159}],
 [{'end': 93, 'label': 'PERSON', 'start': 81},
  {'end': 158, 'label': 'ORG', 'start': 151},
  {'end': 217, 'label': 'ORG', 'start': 196},
  {'end': 233, 'label': 'DATE', 'start': 228}],
 [{'end': 39, 'label': 'DATE', 'start': 20},
  {'end': 314, 'label': 'PERSON', 'start': 302}],
 [{'end': 35, 'label': 'PERSON', 'start': 24},
  {'end': 57, 'label': 'PRODUCT', 'start': 39},
  {'end': 120, 'label': 'PRODUCT', 'start': 88},
  {'end': 153, 'label': 'PRODUCT', 'start': 139}],
 [{'end': 112, 'label': 'CARDINAL', 'start': 108},
  {'end': 121, 'label': 'ORDINAL', 'start': 116},
  {'end': 226, 'label': 'ORDINAL', 'start': 220},
  {'end': 336, 'label': 'ORDINAL', 'start': 331},
  {'end': 446, 'label': 'ORDINAL', 'start': 440},
  {'end': 458, 'label': 'ORDINAL',

In [23]:
for i in name_entity_label_risultati:
  if(len(i)== 0):
    name_entity_label_risultati.remove(i)
name_entity_label_risultati   

[[{'end': 14, 'label': 'PRODUCT', 'start': 1},
  {'end': 142, 'label': 'ORG', 'start': 136},
  {'end': 187, 'label': 'DATE', 'start': 176}],
 [{'end': 94, 'label': 'PERSON', 'start': 81},
  {'end': 181, 'label': 'ORG', 'start': 159}],
 [{'end': 93, 'label': 'ORG', 'start': 81},
  {'end': 158, 'label': 'PRODUCT', 'start': 151},
  {'end': 217, 'label': 'ORG', 'start': 196},
  {'end': 233, 'label': 'DATE', 'start': 228}],
 [{'end': 39, 'label': 'DATE', 'start': 20},
  {'end': 314, 'label': 'ORG', 'start': 302}],
 [{'end': 35, 'label': 'PERSON', 'start': 24},
  {'end': 57, 'label': 'PRODUCT', 'start': 39},
  {'end': 120, 'label': 'PRODUCT', 'start': 88},
  {'end': 153, 'label': 'PRODUCT', 'start': 139}],
 [{'end': 112, 'label': 'CARDINAL', 'start': 108},
  {'end': 121, 'label': 'ORDINAL', 'start': 116},
  {'end': 226, 'label': 'ORDINAL', 'start': 220},
  {'end': 336, 'label': 'ORDINAL', 'start': 331},
  {'end': 446, 'label': 'ORDINAL', 'start': 440},
  {'end': 458, 'label': 'ORDINAL', 'sta

### Valutazione NER (quante label riconosce e quante ne avrebbe dovute trovare)

In [49]:
#df_ner_corretto
df_numero_trovati = pd.read_csv("trovate_effettive.csv")
df_numero_trovati.index += 1 
df_ner_corretto_totale = df_ner_corretto.join(df_numero_trovati)
df_ner_corretto_totale

,phrase,label,trovate,effettive
1,"The Dark Mark had been set over it — well, f...","[{'label': 'PRODUCT', 'start': 1, 'end': 14}, ...",3,5
2,"It was a pompous little sign, neatly lettere...","[{'label': 'PERSON', 'start': 81, 'end': 94}, ...",2,4
3,” Harry held the paper up to the candlelight...,"[{'label': 'PERSON', 'start': 81, 'end': 93}, ...",4,5
4,Harry spent the long sunny days explor...,"[{'label': 'DATE', 'start': 20, 'end': 39}, {'...",2,4
5,What she did have were Bertie Bott’s Every F...,"[{'label': 'PERSON', 'start': 24, 'end': 35}, ...",4,7
...,...,...,...,...
107,“Come to throw something else at me?” Harr...,"[{'label': 'PERSON', 'start': 141, 'end': 147}]",1,2
108,"” Aragog clicked his pincers furiously, and ...","[{'label': ‘PERSON', 'start': 4, 'end': 10}]",3,5
109,For nearly a quarter of an hour he spoke int...,"[{'label': 'DATE', 'start': 5, 'end': 22}, {'l...",1,1
110,"The back, under what looked like a dirty bro...","[{'label': ‘PERSON', 'start': 132, 'end': 137}]",3,4


In [54]:
print("Il numero di label trovate è :" , df_ner_corretto_totale.trovate.sum())
print("Il numero di label che avrebbe dovuto trovare è :" , df_ner_corretto_totale.effettive.sum())
print("Percentuale identificate : ", "{:.2f}".format((df_ner_corretto_totale.trovate.sum()/df_ner_corretto_totale.effettive.sum())*100) )
print("Errore del : ", "{:.2f}".format((100-((df_ner_corretto_totale.trovate.sum()/df_ner_corretto_totale.effettive.sum())*100))) )


Il numero di label trovate è : 332
Il numero di label che avrebbe dovuto trovare è : 463
Percentuale identificate :  71.71
Errore del :  28.29


### Valutazione NER(delle label trovate quante sono corrette) 

In [24]:
true = ner_corretto

pred = name_entity_label_risultati

from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=['PRODUCT', 'PERSON','EVENT','QUANTITY','MONEY', 'ORG', 'CARDINAL','FAC','LOC', 'GPE', 'TIME', 'WORK_OF_ART','ORDINAL','DATE'])

# Risultati delle metriche per ogni tag 

results, results_per_tag = evaluator.evaluate()

print("PRODUCT" , results_per_tag["PRODUCT"] )
print("PERSON" ,results_per_tag["PERSON"] )
print("EVENT" , results_per_tag["EVENT"] )
print("MONEY" , results_per_tag["MONEY"] )
print("QUANTITY", results_per_tag["QUANTITY"] )
print("CARDINAL", results_per_tag["CARDINAL"] )
print("FAC", results_per_tag["FAC"] )
print("LOC" , results_per_tag["LOC"] )
print("GPE", results_per_tag["GPE"] )
print("TIME", results_per_tag["TIME"] )
print("WORK_OF_ART", results_per_tag["WORK_OF_ART"] )
print("ORDINAL", results_per_tag["ORDINAL"] )
print("DATE", results_per_tag["DATE"] )



PRODUCT {'ent_type': {'correct': 7, 'incorrect': 7, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 14, 'actual': 14, 'precision': 0.5, 'recall': 0.5, 'f1': 0.5}, 'partial': {'correct': 14, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 14, 'actual': 14, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'strict': {'correct': 7, 'incorrect': 7, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 14, 'actual': 14, 'precision': 0.5, 'recall': 0.5, 'f1': 0.5}, 'exact': {'correct': 14, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 14, 'actual': 14, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}
PERSON {'ent_type': {'correct': 193, 'incorrect': 33, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 226, 'actual': 226, 'precision': 0.8539823008849557, 'recall': 0.8539823008849557, 'f1': 0.8539823008849556}, 'partial': {'correct': 226, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 226, 'actual': 226, 'precision': 1.0, '

## Named Entity Linking

In [25]:
nlp3 = spacy.load('en_core_web_lg')
string_name = []
link = []
nlp3.add_pipe('dbpedia_spotlight')
for x in range(df.shape[0]):
    print(x)
    doc = nlp3(df["phrase"].values[x])
    annotations =  [(ent.text, ent.label_, ent.kb_id_) for ent in doc.ents]
    for substring, label, url in annotations:
        new_sentence = df["phrase"].values[x].replace(substring, url)
        string_name.append(substring)
        link.append(url)
        print(substring,url)

0
Dark Mark http://dbpedia.org/resource/Magic_in_Harry_Potter
Sirius http://dbpedia.org/resource/Sirius
Regulus http://dbpedia.org/resource/Regulus
1
Percy Weasley http://dbpedia.org/resource/Ministry_of_Magic
Regulus Arcturus Black http://dbpedia.org/resource/Death_Eater
2
STILL AT LARGE http://dbpedia.org/resource/Still_at_Large
Black http://dbpedia.org/resource/African_Americans
Azkaban http://dbpedia.org/resource/Places_in_Harry_Potter
3
sunny days http://dbpedia.org/resource/Dou_Wei
Black http://dbpedia.org/resource/African_Americans
Azkaban http://dbpedia.org/resource/Places_in_Harry_Potter
4
Bott http://dbpedia.org/resource/Raoul_Bott
Pasties http://dbpedia.org/resource/Pasty
Wands http://dbpedia.org/resource/Suit_of_wands
5
forevermore http://dbpedia.org/resource/Forevermore_(TV_series)
four 
First 
poison http://dbpedia.org/resource/Poison
nettle http://dbpedia.org/resource/Urtica_dioica
wine http://dbpedia.org/resource/Wine
Second 
Third 
Fourth 
second 
second 
first 
sight 

In [26]:
risultati_nel = pd.DataFrame(
    {'string_name': string_name,
     'link': link
    })
risultati_nel



,string_name,link
0,Dark Mark,http://dbpedia.org/resource/Magic_in_Harry_Potter
1,Sirius,http://dbpedia.org/resource/Sirius
2,Regulus,http://dbpedia.org/resource/Regulus
3,Percy Weasley,http://dbpedia.org/resource/Ministry_of_Magic
4,Regulus Arcturus Black,http://dbpedia.org/resource/Death_Eater
...,...,...
420,yer,http://dbpedia.org/resource/Yer
421,Invisibility Cloak,http://dbpedia.org/resource/Magical_objects_in...
422,Hermione,http://dbpedia.org/resource/Hermione_Granger
423,Hagrid,http://dbpedia.org/resource/Rubeus_Hagrid


### Creazione dataframe contentente i dati corretti di DBpedia tramite query SPARQL

In [27]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_country_description(query):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
     
    sparql.setQuery(query) 

    return sparql.query().convert()

In [28]:
characters_list = pd.read_csv("characters_list.csv", sep=";")
characters_entity = characters_list.Entity
characters_list = characters_list.name
characters_list

0     Regulus Arcturus Black
1               Sirius Black
2             Lavender Brown
3                  Cho Chang
4             Vincent Crabbe
               ...          
57                    Fluffy
58                    Hedwig
59            Moaning Myrtle
60                    Aragog
61                     Grawp
Name: name, Length: 62, dtype: object

In [29]:
personaggi= []
url_dbpedia = []
for i in characters_list:
  query = """SELECT ?link
  WHERE
  { 
    ?link rdfs:label '"""+i+"""'@en.
  }
  """
  data=get_country_description(query)
  if(len(data["results"]["bindings"]) > 0):
      print(i, data["results"]["bindings"][0]["link"]["value"])
      personaggi.append(i)
      url_dbpedia.append(data["results"]["bindings"][0]["link"]["value"])
  else:
    print(i, data["results"]["bindings"])

Regulus Arcturus Black http://dbpedia.org/resource/Regulus_Arcturus_Black
Sirius Black http://dbpedia.org/resource/Sirius_Black
Lavender Brown http://dbpedia.org/resource/Lavender_Brown
Cho Chang http://dbpedia.org/resource/Cho_Chang
Vincent Crabbe http://dbpedia.org/resource/Vincent_Crabbe
Bartemius "Barty" Crouch Sr. []
Bartemius "Barty" Crouch Jr. []
Fleur Delacour http://dbpedia.org/resource/Fleur_Delacour
Cedric Diggory http://dbpedia.org/resource/Cedric_Diggory
Aberforth Dumbledore http://dbpedia.org/resource/Aberforth_Dumbledore
Albus Dumbledore http://dbpedia.org/resource/Albus_Dumbledore
Dudley Dursley http://dbpedia.org/resource/Dudley_Dursley
Petunia Dursley http://dbpedia.org/resource/Petunia_Dursley
Vernon Dursley http://dbpedia.org/resource/Vernon_Dursley
Argus Filch http://dbpedia.org/resource/Argus_Filch
Seamus Finnigan http://dbpedia.org/resource/Seamus_Finnigan
Nicolas Flamel http://dbpedia.org/resource/Nicolas_Flamel
Cornelius Fudge http://dbpedia.org/resource/Cornel

In [30]:
characters_entity = characters_entity.drop([5,6])


In [39]:
risultati_nel_corretti = pd.DataFrame(
    {'name': personaggi,
     'link_dbpedia': url_dbpedia,
     'entity' : characters_entity
    })
risultati_nel_corretti.to_csv("risultati_nel_corretti.csv")
risultati_nel_corretti


,name,link_dbpedia,entity
0,Regulus Arcturus Black,http://dbpedia.org/resource/Regulus_Arcturus_B...,Regulus
1,Sirius Black,http://dbpedia.org/resource/Sirius_Black,Sirius
2,Lavender Brown,http://dbpedia.org/resource/Lavender_Brown,Lavander
3,Cho Chang,http://dbpedia.org/resource/Cho_Chang,Cho
4,Vincent Crabbe,http://dbpedia.org/resource/Vincent_Crabbe,Crabbe
7,Fleur Delacour,http://dbpedia.org/resource/Fleur_Delacour,Fleur
8,Cedric Diggory,http://dbpedia.org/resource/Cedric_Diggory,Cedric
9,Aberforth Dumbledore,http://dbpedia.org/resource/Aberforth_Dumbledore,Aberforth
10,Albus Dumbledore,http://dbpedia.org/resource/Albus_Dumbledore,Dumbledore
11,Dudley Dursley,http://dbpedia.org/resource/Dudley_Dursley,Dubley


### 1. Valutazione Nel personaggi - più precisa (si controlla solamente il nome)

In [40]:
df = risultati_nel.groupby(['string_name','link']).size().reset_index(name='Freq')
df



,string_name,link,Freq
0,ALBUS DUMBLEDORE,http://dbpedia.org/resource/Albus_Dumbledore,1
1,Aberforth,http://dbpedia.org/resource/Order_of_the_Phoen...,2
2,Agrippa,http://dbpedia.org/resource/Heinrich_Cornelius...,1
3,Aidan,http://dbpedia.org/resource/Aidan_of_Lindisfarne,1
4,Albania,http://dbpedia.org/resource/Albania,1
...,...,...,...
243,witchcraft,http://dbpedia.org/resource/Witchcraft,1
244,wizard,http://dbpedia.org/resource/Unseen_University,4
245,years,,1
246,yeh,,1


In [41]:
correct_entity = list(risultati_nel_corretti.entity)
correct_name = list(risultati_nel_corretti.name)


In [42]:
count_totale = 0
count_esatti = 0
count_sbagliati = 0
count_non_riconosciuti = 0
for index,row in df.iterrows():
  if((row["string_name"].title() in correct_entity) or(row["string_name"].title() in correct_name)):
    count_totale = count_totale+1
    for index2,row2 in risultati_nel_corretti.iterrows():
      if((row["string_name"].title() == row2["entity"]) or (row["string_name"].title() == row2["name"])):
        print(row["string_name"], row["link"],"-", row2["link_dbpedia"])
        if((len(row["link"]) > 0) and (row["link"] == row2["link_dbpedia"])):
          count_esatti = count_esatti +1
        else: 
          if(len(row["link"]) > 0):
            count_sbagliati = count_sbagliati + 1
          else: count_non_riconosciuti= count_non_riconosciuti +1

print("Totale personaggi trovati da NEL: ", count_totale)
print("Totale personaggi riconosciuti correttamente: ", count_esatti)
print("Totale personaggi riconosciuti con un diverso link: ", count_sbagliati)
print("Totale personaggi non riconosciuti da NEL: ", count_non_riconosciuti)

ALBUS DUMBLEDORE http://dbpedia.org/resource/Albus_Dumbledore - http://dbpedia.org/resource/Albus_Dumbledore
Aberforth http://dbpedia.org/resource/Order_of_the_Phoenix_(fictional_organisation) - http://dbpedia.org/resource/Aberforth_Dumbledore
Alice Longbottom http://dbpedia.org/resource/Order_of_the_Phoenix_(fictional_organisation) - http://dbpedia.org/resource/Alice_Longbottom
Aragog http://dbpedia.org/resource/Magical_creatures_in_Harry_Potter - http://dbpedia.org/resource/Aragog
Aragog http://dbpedia.org/resource/Rubeus_Hagrid - http://dbpedia.org/resource/Aragog
Argus Filch http://dbpedia.org/resource/Hogwarts_staff - http://dbpedia.org/resource/Argus_Filch
Arthur http://dbpedia.org/resource/King_Arthur - http://dbpedia.org/resource/Arthur_Weasley
Bellatrix http://dbpedia.org/resource/Death_Eater - http://dbpedia.org/resource/Bellatrix_Lestrange
Bill  - http://dbpedia.org/resource/Bill_Weasley
Cedric Diggory  - http://dbpedia.org/resource/Cedric_Diggory
Charlie  - http://dbpedia.o

### 2. Valutazione Nel Personaggi - meno specifica (si controlla che il risultato di NEL appartenga ad Harry Potter)

In [43]:
risultati_nel_corretti_2 = pd.read_csv("risultati_nel_corretti_2.csv")
risultati_nel_corretti_2 = risultati_nel_corretti_2.iloc[:,1:]
risultati_nel_corretti_2["film"] = "http://dbpedia.org/resource/Order_of_the_Phoenix_(fictional_organisation)"
risultati_nel_corretti_2

,name,link_dbpedia,entity,second_link,film
0,Regulus Arcturus Black,http://dbpedia.org/resource/Regulus_Arcturus_B...,Regulus,http://dbpedia.org/resource/Death_Eater,http://dbpedia.org/resource/Order_of_the_Phoen...
1,Sirius Black,http://dbpedia.org/resource/Sirius_Black,Sirius,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
2,Lavender Brown,http://dbpedia.org/resource/Lavender_Brown,Lavander,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
3,Cho Chang,http://dbpedia.org/resource/Cho_Chang,Cho,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
4,Vincent Crabbe,http://dbpedia.org/resource/Vincent_Crabbe,Crabbe,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
5,Fleur Delacour,http://dbpedia.org/resource/Fleur_Delacour,Fleur,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
6,Cedric Diggory,http://dbpedia.org/resource/Cedric_Diggory,Cedric,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
7,Aberforth Dumbledore,http://dbpedia.org/resource/Aberforth_Dumbledore,Aberforth,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
8,Albus Dumbledore,http://dbpedia.org/resource/Albus_Dumbledore,Dumbledore,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...
9,Dudley Dursley,http://dbpedia.org/resource/Dudley_Dursley,Dubley,NaN,http://dbpedia.org/resource/Order_of_the_Phoen...


In [44]:
count_totale = 0
count_esatti = 0
count_sbagliati = 0
count_non_riconosciuti = 0
for index,row in df.iterrows():
  if((row["string_name"].title() in correct_entity) or(row["string_name"].title() in correct_name)):
    count_totale = count_totale+1
    for index2,row2 in risultati_nel_corretti_2.iterrows():
      if((row["string_name"].title() == row2["entity"]) or (row["string_name"].title() == row2["name"])):
        print(row["string_name"], row["link"],"-", row2["link_dbpedia"],row2["second_link"] )
        if((len(row["link"]) > 0) and (row["link"] == row2["link_dbpedia"] or row["link"] == row2["second_link"] or row["link"] == row2["film"] )):
          count_esatti = count_esatti +1
        else:
          if(len(row["link"]) > 0):
            count_sbagliati = count_sbagliati + 1
          else: count_non_riconosciuti= count_non_riconosciuti +1

print("Totale personaggi trovati da NEL: ", count_totale)
print("Totale personaggi riconosciuti correttamente: ", count_esatti)
print("Totale personaggi riconosciuti con un diverso link: ", count_sbagliati)
print("Totale personaggi non riconosciuti da NEL : ", count_non_riconosciuti)

ALBUS DUMBLEDORE http://dbpedia.org/resource/Albus_Dumbledore - http://dbpedia.org/resource/Albus_Dumbledore nan
Aberforth http://dbpedia.org/resource/Order_of_the_Phoenix_(fictional_organisation) - http://dbpedia.org/resource/Aberforth_Dumbledore nan
Alice Longbottom http://dbpedia.org/resource/Order_of_the_Phoenix_(fictional_organisation) - http://dbpedia.org/resource/Alice_Longbottom nan
Aragog http://dbpedia.org/resource/Magical_creatures_in_Harry_Potter - http://dbpedia.org/resource/Aragog  http://dbpedia.org/resource/Magical_creatures_in_Harry_Potter
Aragog http://dbpedia.org/resource/Rubeus_Hagrid - http://dbpedia.org/resource/Aragog  http://dbpedia.org/resource/Magical_creatures_in_Harry_Potter
Argus Filch http://dbpedia.org/resource/Hogwarts_staff - http://dbpedia.org/resource/Argus_Filch nan
Arthur http://dbpedia.org/resource/King_Arthur - http://dbpedia.org/resource/Arthur_Weasley nan
Bellatrix http://dbpedia.org/resource/Death_Eater - http://dbpedia.org/resource/Bellatrix_L